In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# We force numpy to a version < 2.0 to fix compatibility issues
!pip install -q "numpy<2.0"

!pip install -q albumentations
!pip install -q segmentation-models-pytorch
!pip install -q tqdm

In [ ]:
import torch
from torch.utils.data import Dataset
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# --- 1. Define the Dataset Class ---

class ForgeryDataset(Dataset):
    """
    A PyTorch Dataset class to load forged images and their .npy masks.
    """
    def __init__(self, images_dir, masks_dir):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        
        # Get all the image filenames (e.g., '47513.png')
        self.image_files = os.listdir(images_dir)

    def __len__(self):
        # This returns the total number of samples
        return len(self.image_files)

    def __getitem__(self, idx):
        # This loads one sample (image + mask)
        
        # 1. Get the filename
        image_filename = self.image_files[idx]
        file_stem = image_filename.split('.')[0]
        mask_filename = file_stem + ".npy"

        # 2. Define the full paths
        img_path = os.path.join(self.images_dir, image_filename)
        mask_path = os.path.join(self.masks_dir, mask_filename)

        # 3. Load the image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # 4. Load the mask
        mask = np.load(mask_path)
        mask = np.squeeze(mask) # Squeeze from (1, H, W) to (H, W)
        
        # In deep learning, masks should be (Height, Width, Channels)
        # We need to add a channel dimension at the end: (H, W) -> (H, W, 1)
        mask = np.expand_dims(mask, axis=-1)

        # --- IMPORTANT ---
        # Normalize image values from [0, 255] to [0.0, 1.0]
        img = img.astype(np.float32) / 255.0
        # Normalize mask values from [0, 255] to [0.0, 1.0] (for segmentation)
        mask = mask.astype(np.float32) / 255.0
        
        # Convert to PyTorch tensors
        # (H, W, C) -> (C, H, W) as PyTorch expects
        img_tensor = torch.from_numpy(img).permute(2, 0, 1)
        mask_tensor = torch.from_numpy(mask).permute(2, 0, 1)

        return img_tensor, mask_tensor

# --- 2. Test the Dataset Class ---

# Define your paths again
BASE_PATH = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
FORGED_IMG_PATH = os.path.join(BASE_PATH, "train_images", "forged")
TRAIN_MASK_PATH = os.path.join(BASE_PATH, "train_masks")

# Create an instance of your dataset
train_dataset = ForgeryDataset(images_dir=FORGED_IMG_PATH, masks_dir=TRAIN_MASK_PATH)

# Let's load the 5th item (index 4) from the dataset
img, mask = train_dataset[4]

print(f"Total images in dataset: {len(train_dataset)}")
print(f"Loaded image tensor shape: {img.shape}")  # Should be [3, H, W]
print(f"Loaded mask tensor shape:  {mask.shape}") # Should be [1, H, W]

# --- 3. Visualize the Tensors ---
# We need to convert them back to (H, W, C) for plotting
img_to_show = img.permute(1, 2, 0).numpy()
mask_to_show = mask.permute(1, 2, 0).squeeze().numpy() # .squeeze() to remove the 1

plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
plt.imshow(img_to_show)
plt.title("Loaded Image (as Tensor)")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(mask_to_show, cmap='gray')
plt.title("Loaded Mask (as Tensor)")
plt.axis('off')

plt.show()

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# 1. Define your data paths
BASE_PATH = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
FORGED_IMG_PATH = os.path.join(BASE_PATH, "train_images", "forged")
TRAIN_MASK_PATH = os.path.join(BASE_PATH, "train_masks")

# 2. Get a list of all FORGED images
image_files = os.listdir(FORGED_IMG_PATH)
print(f"Found {len(image_files)} forged images.")
print("---")

# Let's try a different image (e.g., the 10th one)
image_to_check = image_files[10] 
print(f"Attempting to load image: {image_to_check}")

# 3. Define the full paths
img_path = os.path.join(FORGED_IMG_PATH, image_to_check)
mask_path = os.path.join(TRAIN_MASK_PATH, image_to_check)

# --- DEBUGGING CHECK ---
# Let's check if the mask file path we built is correct.
if not os.path.exists(mask_path):
    print(f"ERROR: The assumed mask path does NOT exist:")
    print(f"  {mask_path}")
    print("\nThis means the mask filenames do not match the image filenames.")
    
    # Let's see what the mask filenames actually look like:
    print("\n--- Listing first 5 files in train_masks: ---")
    mask_files_list = os.listdir(TRAIN_MASK_PATH)
    print(mask_files_list[:5])
    print("-------------------------------------------------")
    print("Compare the list above to the image name. You may need to change the filename.")

else:
    # 4. Load the image and its mask
    print("SUCCESS: Found matching mask file!")
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if mask is None:
        print(f"ERROR: Mask file at {mask_path} exists, but cv2.imread failed. File may be corrupt.")
    else:
        # 5. Display them side-by-side
        plt.figure(figsize=(15, 7))

        plt.subplot(1, 2, 1)
        plt.imshow(img)
        plt.title(f"Original Forged Image\n({image_to_check})")
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(mask, cmap='gray')
        plt.title(f"Forgery Mask\n({image_to_check})")
        plt.axis('off')

        plt.show()

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

# 1. Define your data paths
BASE_PATH = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
FORGED_IMG_PATH = os.path.join(BASE_PATH, "train_images", "forged")
TRAIN_MASK_PATH = os.path.join(BASE_PATH, "train_masks")

# 2. Get a list of all FORGED images
image_files = os.listdir(FORGED_IMG_PATH)
print(f"Found {len(image_files)} forged images.")
print("---")

# Let's pick an image to check (e.g., the 10th one)
image_filename = image_files[10] # This will be like '47513.png'

# 3. Build the correct .npy filename
file_stem = image_filename.split('.')[0]
mask_filename = file_stem + ".npy"

print(f"Loading image: {image_filename}")
print(f"Loading mask:  {mask_filename}")

# 4. Define the full paths
img_path = os.path.join(FORGED_IMG_PATH, image_filename)
mask_path = os.path.join(TRAIN_MASK_PATH, mask_filename)

# 5. Load the image and its mask
# Load the image with cv2
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

# Load the .npy file
mask = np.load(mask_path)
print(f"Original mask shape: {mask.shape}")

# --- THIS IS THE FIX ---
# Squeeze the extra '1' dimension out
mask = np.squeeze(mask)
print(f"Squeezed mask shape: {mask.shape}") # This will now be (68, 875)
# ----------------------

# 6. Display them side-by-side
plt.figure(figsize=(15, 7))

# Subplot for the Original Image
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title(f"Original Forged Image\n({image_filename})")
plt.axis('off')

# Subplot for the Forgery Mask
plt.subplot(1, 2, 2)
plt.imshow(mask, cmap='gray') # This will now work
plt.title(f"Forgery Mask (The 'Answer')\n({mask_filename})")
plt.axis('off')

plt.show()

In [ ]:
# --- 1. Install required libraries ---
!pip install -q albumentations
!pip install -q segmentation-models-pytorch
!pip install -q tqdm  # For a progress bar

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import cv2
import numpy as np
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from tqdm import tqdm # Import tqdm

# --- 2. Update the Dataset Class (The Fix is Here) ---
class ForgeryDataset(Dataset):
    def __init__(self, images_dir, masks_dir, augmentations=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.augmentations = augmentations
        
        # --- DATA INTEGRITY CHECK ---
        # We will check all files and only keep the ones where
        # image and mask shapes match.
        print("Running data integrity check...")
        all_image_files = os.listdir(images_dir)
        self.image_files = [] # This will be our "clean" list
        
        for image_filename in tqdm(all_image_files): # Use tqdm for a progress bar
            try:
                # Get paths
                file_stem = image_filename.split('.')[0]
                mask_filename = file_stem + ".npy"
                img_path = os.path.join(self.images_dir, image_filename)
                mask_path = os.path.join(self.masks_dir, mask_filename)

                # Check if mask file exists
                if not os.path.exists(mask_path):
                    continue # Skip if no matching mask

                # Load shapes
                img = cv2.imread(img_path)
                img_shape = img.shape[:2] # (H, W)
                
                mask = np.load(mask_path)
                mask_shape = np.squeeze(mask).shape # (H, W)
                
                # Check for match
                if img_shape == mask_shape:
                    self.image_files.append(image_filename) # Add to our clean list
            except Exception as e:
                # Catch any other loading errors (like corrupt files)
                print(f"\nWarning: Skipping {image_filename} due to error: {e}")
        
        print(f"Integrity check complete. Found {len(self.image_files)} valid image/mask pairs.")

    def __len__(self):
        # This will now return the count of *clean* files
        return len(self.image_files)

    def __getitem__(self, idx):
        # 1. Get filenames (from our clean list)
        image_filename = self.image_files[idx]
        file_stem = image_filename.split('.')[0]
        mask_filename = file_stem + ".npy"

        # 2. Define paths
        img_path = os.path.join(self.images_dir, image_filename)
        mask_path = os.path.join(self.masks_dir, mask_filename)

        # 3. Load image and mask (as NumPy arrays)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        mask = np.load(mask_path)
        mask = np.squeeze(mask) # Squeeze from (1, H, W) to (H, W)
        
        # 4. Apply Augmentations
        if self.augmentations:
            augmented = self.augmentations(image=img, mask=mask)
            img = augmented['image']
            mask = augmented['mask']
        
        # 5. Process Mask (using PyTorch operations)
        mask = mask.to(torch.float32) / 255.0
        mask = mask.unsqueeze(0)
        
        return img, mask

# --- 3. Define Augmentations ---
IMG_SIZE = 384
BATCH_SIZE = 8 

train_augs = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2() 
])

val_augs = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# --- 4. Create and Split the Datasets ---
BASE_PATH = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
FORGED_IMG_PATH = os.path.join(BASE_PATH, "train_images", "forged")
TRAIN_MASK_PATH = os.path.join(BASE_PATH, "train_masks")

# This will now run the integrity check
full_dataset = ForgeryDataset(FORGED_IMG_PATH, TRAIN_MASK_PATH)

# Split it into 90% training, 10% validation
train_size = int(0.9 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_data, val_data = random_split(full_dataset, [train_size, val_size])

# Apply the different augmentations to the two splits
train_data.dataset.augmentations = train_augs
val_data.dataset.augmentations = val_augs

# --- 5. Create DataLoaders ---
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"Total valid images: {len(full_dataset)}")
print(f"Training images: {len(train_data)}")
print(f"Validation images: {len(val_data)}")
print(f"Batch size: {BATCH_SIZE}")

# --- 6. Test the DataLoader ---
# This should now work perfectly!
images, masks = next(iter(train_loader))

print("\n--- Test Batch Shapes ---")
print(f"Images batch shape: {images.shape}")
print(f"Masks batch shape:  {masks.shape}")

print("\nData pipeline is 100% complete and ready for training! ✅")

In [ ]:
import torch  # <-- THIS IS THE FIX
import segmentation_models_pytorch as smp
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# --- 1. Define Model, Loss, and Optimizer ---

# Set the device to GPU if available
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Our "Winning Edge" Model
# U-Net with a pre-trained EfficientNet-B4 backbone
model = smp.Unet(
    encoder_name="efficientnet-b4",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1, # We output 1 channel (the mask)
).to(DEVICE)

# Our "Winning Edge" Loss Function
# A combo of BCE (pixel-level) and Dice (blob-level)
class ComboLoss(nn.Module):
    def __init__(self, bce_weight=0.5, dice_weight=0.5):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss()
        self.dice = smp.losses.DiceLoss(mode='binary')
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight

    def forward(self, inputs, targets):
        bce_loss = self.bce(inputs, targets)
        dice_loss = self.dice(inputs, targets)
        return (self.bce_weight * bce_loss) + (self.dice_weight * dice_loss)

loss_fn = ComboLoss().to(DEVICE)
optimizer = optim.AdamW(model.parameters(), lr=1e-4) # AdamW is a great default
scaler = torch.amp.GradScaler('cuda') # Use the updated torch.amp syntax

# --- 2. Define the Training & Validation Functions ---

def train_fn(loader, model, optimizer, loss_fn, scaler):
    """ Trains the model for one epoch. """
    model.train() # Set the model to training mode
    loop = tqdm(loader, desc="Training")
    
    for images, masks in loop:
        # Move data to the GPU
        images = images.to(DEVICE, dtype=torch.float32)
        masks = masks.to(DEVICE, dtype=torch.float32)
        
        # Forward pass with mixed precision
        with torch.amp.autocast('cuda'):
            predictions = model(images)
            loss = loss_fn(predictions, masks)
            
        # Backward pass
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        loop.set_postfix(loss=loss.item())

def val_fn(loader, model, loss_fn):
    """ Validates the model. """
    model.eval() # Set the model to evaluation mode
    loop = tqdm(loader, desc="Validation")
    val_loss = 0
    
    with torch.no_grad(): # No gradients needed for validation
        for images, masks in loop:
            images = images.to(DEVICE, dtype=torch.float32)
            masks = masks.to(DEVICE, dtype=torch.float32)
            
            with torch.amp.autocast('cuda'):
                predictions = model(images)
                loss = loss_fn(predictions, masks)
                
            val_loss += loss.item()
            loop.set_postfix(loss=loss.item())
            
    avg_loss = val_loss / len(loader)
    print(f"Average Validation Loss: {avg_loss:.4f}")
    return avg_loss

# --- 3. The Training Loop ---

NUM_EPOCHS = 5 # Start with 5 epochs to see how it goes
best_val_loss = float("inf")

for epoch in range(NUM_EPOCHS):
    print(f"\n--- Epoch {epoch+1}/{NUM_EPOCHS} ---")
    
    train_fn(train_loader, model, optimizer, loss_fn, scaler)
    val_loss = val_fn(val_loader, model, loss_fn)
    
    # Save the model if it's the best one so far
    if val_loss < best_val_loss:
        print(f"Validation loss improved! Saving model...")
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")

print("\n--- Training Complete ---")
print(f"Best Validation Loss: {best_val_loss:.4f}")
print("Model saved to 'best_model.pth'")

In [ ]:
import pandas as pd
import json
import numba
import numpy.typing as npt
import torchvision.transforms.functional as F # This is for transforms, NOT interpolate
import torch.nn.functional as F_nn           # <-- THIS IS THE FIX (Import)
from torch.utils.data import Dataset, DataLoader

# --- 1. Copy the RLE Encoding Functions ---
@numba.jit(nopython=True)
def _rle_encode_jit(x: npt.NDArray, fg_val: int = 1) -> list[int]:
    """Numba-jitted RLE encoder."""
    dots = np.where(x.T.flatten() == fg_val)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def rle_encode(masks: list[npt.NDArray], fg_val: int = 1) -> str:
    """
    Encodes a list of masks into a single RLE string.
    We pass a list with one mask: [my_mask]
    """
    return ';'.join([json.dumps(_rle_encode_jit(x, fg_val)) for x in masks])

# --- 2. Create the Test Dataset ---
class TestDataset(Dataset):
    def __init__(self, images_dir, augmentations):
        self.images_dir = images_dir
        self.image_files = os.listdir(images_dir)
        self.augmentations = augmentations

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_filename = self.image_files[idx]
        image_id = image_filename.split('.')[0]
        
        img_path = os.path.join(self.images_dir, image_filename)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        original_shape = img.shape[:2] # (H, W) - This is a tuple
        
        augmented = self.augmentations(image=img)
        img_tensor = augmented['image']
        
        # Return the original shape as-is
        return img_tensor, image_id, original_shape

# --- 3. Define Inference Transforms ---
TEST_IMG_SIZE = 384
test_augs = A.Compose([
    A.Resize(TEST_IMG_SIZE, TEST_IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# --- 4. Create Test DataLoader ---
# Re-using BASE_PATH, BATCH_SIZE, and DEVICE from the previous cell
TEST_IMG_PATH = os.path.join(BASE_PATH, "test_images")
test_dataset = TestDataset(TEST_IMG_PATH, test_augs)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# --- 5. Load Best Model ---
print("Loading best model from 'best_model.pth'...")
model = smp.Unet(
    encoder_name="efficientnet-b4",
    encoder_weights=None,
    in_channels=3,
    classes=1,
).to(DEVICE)

model.load_state_dict(torch.load("best_model.pth"))
model.eval() 

# --- 6. The "Winning Edge" Post-Processing Rules ---
PREDICTION_THRESHOLD = 0.5
MIN_MASK_AREA = 100         

# --- 7. Generate Predictions ---
submission = [] 

with torch.no_grad():
    loop = tqdm(test_loader, desc="Generating Predictions")
    for img_tensor, image_ids, original_shapes in loop:
        
        img_tensor = img_tensor.to(DEVICE, dtype=torch.float32)
        
        with torch.amp.autocast('cuda'):
            pred_masks_small = model(img_tensor)
            
        for i in range(pred_masks_small.shape[0]):
            image_id = image_ids[i]
            
            original_h = original_shapes[0][i]
            original_w = original_shapes[1][i]
            
            # --- THIS IS THE FIX (Call) ---
            pred_mask = F_nn.interpolate(
                pred_masks_small[i].unsqueeze(0), 
                size=(original_h.item(), original_w.item()),
                mode='bilinear', 
                align_corners=False
            ).squeeze()
            
            pred_mask = (torch.sigmoid(pred_mask) > PREDICTION_THRESHOLD).cpu().numpy().astype(np.uint8)

            contours, _ = cv2.findContours(pred_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            final_mask = np.zeros(pred_mask.shape, dtype=np.uint8)
            for cnt in contours:
                if cv2.contourArea(cnt) > MIN_MASK_AREA:
                    cv2.drawContours(final_mask, [cnt], -1, 1, -1)
            
            if np.sum(final_mask) == 0:
                prediction_string = "authentic"
            else:
                prediction_string = rle_encode([final_mask])
                
            submission.append({
                'case_id': image_id,
                'annotation': prediction_string
            })

# --- 8. Create and Save Submission File ---
print("\nSaving submission file...")
submission_df = pd.DataFrame(submission)
submission_df.to_csv("submission.csv", index=False)

print("Done! 🎉 Your submission.csv is ready.")